In [1]:
def print_matrix(matrix):
    return ('[' + 
            ']\n['.join(
                [''.join(
                    ['{}\t'.format(row[i]) if i != len(row)-1 
                     else '{}'.format(row[i]) 
                     for i in range(len(row))]
                ) for row in matrix]
            ) + ']')

In [2]:
t_test = [i/10 for i in range(11)]
x_test = [2, 1.81, 1.64, 1.49, 1.36, 1.25, 1.16, 1.09, 1.04, 1.01, 1]


x0_test = 2
xn_test = 2

# Implementando o cálculo de h

De forma geral, o vetor h é definido da seguinte forma:

> h<sub>i+1</sub> = t<sub>i+1</sub> - t<sub>i</sub>, para i no intervalo \[0, n-1\]

Para facilitar a implementação, o h<sub>0</sub> será considerado 0, mas não será usado na conta.

In [3]:
def calculate_h(values):
    h = [None] + [
        values[i+1] - values[i]
        for i in range(len(values)-1)
    ]
    
    return h

# Calculando λ

Cada um dos λ<sub>i</sub>, i ∈ \[0, n-1\] é descrito da seguinte forma:

> se i = 0 &rarr; λ<sub>0</sub> = 0 <br> senão &rarr; λ<sub>i</sub> = <sup>h<sub>i+1</sub></sup>&frasl;<sub>(h<sub>i</sub> + h<sub>i+1</sub>)

In [4]:
def calculate_lambda(h_values):
    lambda_ = [0.0] + [
        h_values[i+1]/(h_values[i]+h_values[i+1])
        for i in range(1, len(h_values)-1)
    ]
    
    return lambda_

# Calculando μ

Cada um dos μ<sub>i</sub> i ∈ \[1, n\] são definidos da seguinte forma:

> se i == n &rarr; μ<sub>i</sub> = 0 <br> senão μ<sub>i</sub> = <sup>h<sub>i</sub></sup>&frasl;<sub>(h<sub>i</sub> + h<sub>i+1</sub>)

Para efeito de facilidade, definiremos o μ<sub>0</sub> como 0, mas ele não será utilizado na matriz final.

In [5]:
def calculate_mi(h_values):
    mi = [None] + [
        h_values[i]/(h_values[i]+h_values[i+1]) 
        for i in range(1, len(h_values)-1)
    ] + [0.0]
    
    return mi

# Matriz

Com isso já é possível construir a matriz, como segue na função:

In [6]:
def build_matrix(lambda_, mi, size):
    matrix = []
    for i in range(size):
        matrix_i = []
        for j in range(size):
            if i == j:
                matrix_i += [2.0]
            elif i-1 == j:
                matrix_i += [mi[j+1]]
            elif j-1 == i:
                matrix_i += [lambda_[i]]
            else:
                matrix_i += [0.0]
        matrix += [matrix_i]
    
    return matrix

Para efeito de teste, uma das características dessa matriz é ser quadrada. Aqui se encontra uma forma de descobrir se esta matriz é quadrada.


In [7]:
def is_square_matrix(matrix):
    expected_size = len(matrix)
    
    for each_list in matrix:
        if len(each_list) != expected_size:
            return False
        
    return True

## Calculando d

Cada um dos d<sub>i</sub> entre \[0, n\], e dados d<sub>0</sub> e d<sub>n</sub>, o vetor d é o que segue:


In [8]:
def calculate_d(x, h, x0, xn):
    therms = [
        (6 / (h[i] + h[i+1]),
        (x[i+1] - x[i]) / h[i+1],
        (x[i] - x[i-1])/h[i])
        for i in range(1, len(h)-1)
    ]
    
    d = [x0] + [
        therm[0] * (therm[1] - therm[2])
        for therm in therms
    ] + [xn]
    
    return d

Possuindo todos os termos, o objetivo é encontrar o valor das constantes M<sub>i</sub>. Para isso, é necessário calcular a equação Ax = b, onde A é a matriz, e b o vetor d. Para este problema, foi proposto a solução de Gauss e de Jacobi, como segue:

## Eliminação de Gauss
Primeiro deve-se escalonar a matriz, conforme o algoritmo abaixo.

In [9]:
def echelon_form(m, y):
    for j in range(len(m)-1):
        if m[j][j] == 0:
            k = 0
            for k in range(j+1, len(m)):
                if m[k][j] != 0:
                    m[k], m[j] = m[j], m[k]
                    break
            
            if k == len(m):
                return None
        
        for i in range(j+1, len(m)):
            m_i = -m[i][j] / m[j][j]
            
            for k in range(j, len(m)):
                m[i][k] += m_i * m[j][k]
                
            y[i] += m_i * y[j]
            
    
    return m

Depois, devemos calcular o valor da solução, podendo usar o método de eliminação de Gauss, como segue.

In [10]:
def gauss(a, b):
    m_echelon_form = echelon_form(a, b)
    matrix_values = m_echelon_form
    
    __x = [0.0 for i in range(len(matrix_values))]
    
    for i in range(len(matrix_values)-1, -1, -1):
        xi = b[i]
        
        for j in range(i+1, len(matrix_values)):
            xi -= matrix_values[i][j] * __x[j]
        
        xi /= matrix_values[i][i]
        __x[i] = xi
    
    return __x
        

## Algoritmo de Jacobi

Para calcular a solução da matriz usando Jacobi, é necessários critérios de convergência. É comum que se escolha uma tolerância e um número de iterações máximo.

Para o número de iterações, basta contabilizar o número de vezes que um laço foi executado.

Quanto a tolerância, o desejado é que um número epsilon fique menor que a tolerância, e portanto precisa definir um valor de epsilon para cada iteração. Usaremos para esse caso a norma-∞ da diferença do vetor da iteração atual e da iteração anterior, como segue: 

In [11]:
def norm_of_vectors_difference(x, y):
    if len(x) != len(y):
        return None
        
    norms_list = [abs(x[i]-y[i]) for i in range(len(x))]
    return max(norms_list)

In [12]:
def jacobi(d, lambda_, mi, guessed_x, tolerance, max_iteration):
    epsilon = tolerance + 1
    k = 0
    
    x_old = guessed_x.copy()
    x_new = guessed_x.copy()
        
    while k < max_iteration and epsilon > tolerance:
        x0 = (1/2) * (d[0])
        xn = (1/2) * (d[len(d)-1])
        
        x_new = [x0] + [
            (1/2) * (d[i] - mi[i] * x_old[i-1] - lambda_[i] * x_old[i+1])
            for i in range(1, len(d)-1)
        ] + [xn]
              
        epsilon = norm_of_vectors_difference(x_new, x_old)
        x_old = x_new.copy()
        k += 1
    
    return x_new

Agora, só é necessário calcular os valores das constantes A e B

In [13]:
def calculate_a(x, h, m):
    therms = [
        ((x[i+1]-x[i])/h[i+1],
        h[i+1] * (m[i+1]-m[i])/6)
        for i in range(len(x)-1)
    ]
    
    a = [therm_one - therm_two for therm_one, therm_two in therms]
    
    return a
    

In [14]:
def calculate_b(x, h, m):
    b = [
        x[i] - (m[i] / 6) * h[i+1] ** 2
        for i in range(len(x)-1)
    ]
    
    return b

## Calculando para os valores de teste

In [15]:
h = calculate_h(t_test)

lambda_ = calculate_lambda(h)

mi = calculate_mi(h)

matrix = build_matrix(lambda_, mi, len(t_test))
print(print_matrix(matrix))
print()


d = calculate_d(x_test, h, 2*x0_test, 2*xn_test)

gauss_result = gauss([row[:] for row in matrix], d.copy())
jacobi_result = jacobi(d, lambda_, mi, [0]*len(t_test), 1e-8, 1000)
print(f"M_gauss = {gauss_result}\n")
print(f"M_jacob = {jacobi_result}\n")

a_gauss = calculate_a(x_test, h, gauss_result)
a_jacob = calculate_a(x_test, h, jacobi_result)
print(f"A_gauss = {a_gauss}\n")
print(f"A_jacob = {a_jacob}\n")

b_gauss = calculate_b(x_test, h, gauss_result)
b_jacob = calculate_b(x_test, h, jacobi_result)
print(f"B_gauss = {b_gauss}\n")
print(f"B_jacob = {b_jacob}\n")

gauss_result = gauss([row[:] for row in matrix], d.copy())
jacobi_result = jacobi(d, lambda_, mi, [0]*len(t_test), 1e-8, 1000)
print(f"M_gauss = {gauss_result}\n")
print(f"M_jacob = {jacobi_result}\n")

[2.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0]
[0.5	2.0	0.5	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0]
[0.0	0.5000000000000001	2.0	0.49999999999999994	0.0	0.0	0.0	0.0	0.0	0.0	0.0]
[0.0	0.0	0.4999999999999999	2.0	0.5000000000000001	0.0	0.0	0.0	0.0	0.0	0.0]
[0.0	0.0	0.0	0.5000000000000001	2.0	0.4999999999999999	0.0	0.0	0.0	0.0	0.0]
[0.0	0.0	0.0	0.0	0.5	2.0	0.5	0.0	0.0	0.0	0.0]
[0.0	0.0	0.0	0.0	0.0	0.5	2.0	0.5	0.0	0.0	0.0]
[0.0	0.0	0.0	0.0	0.0	0.0	0.4999999999999997	2.0	0.5000000000000002	0.0	0.0]
[0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.5000000000000002	2.0	0.4999999999999997	0.0]
[0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.5	2.0	0.5]
[0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	2.0]

M_gauss = [2.0, 1.9999999999999596, 2.0000000000000395, 2.000000000000014, 1.9999999999999545, 2.0000000000000036, 2.000000000000046, 1.9999999999999645, 2.000000000000002, 2.0000000000000036, 2.0]

M_jacob = [2.0, 1.9999999992687736, 1.9999999985376675, 1.9999999980857468, 1.9999999976337923, 1.9999999976338405, 1.9999999976338836, 1.99999

# Tarefa 2

## 2.1 Monte e resolva o sistema de equações

In [16]:
t = list(range(30))

x = [
    0.0, 1, 2.4, 4.1, 6, 8.2, 10.6, 13.4, 16.4, 19.7,
    23.3, 27, 31.2, 35.5, 40.1, 45, 50.2, 55.6, 61.3, 67.3,
    73.6, 80.1, 86.9, 94, 101.3, 109, 116.9, 125, 133.4, 142.1
]

x0 = (x[2] - x[1])/(t[2] - t[1])
x0 -= (x[1] - x[0])/(t[1] - t[0])
x0 /= (t[2] - t[0])
xn = (x[len(t)-1] - x[len(t)-2])/(t[len(t)-1] - t[len(t)-2])
xn -= (x[len(t)-2] - x[len(t)-3])/(t[len(t)-2] - t[len(t)-3])
xn /= (t[len(t)-1] - t[len(t)-3])

h = calculate_h(t)
mi = calculate_mi(h)
lambda_ = calculate_lambda(h)

matrix = build_matrix(lambda_, mi, len(t))

d = calculate_d(x, h, 2*x0, 2*xn)

## Gauss

In [17]:
gauss_solution = gauss([row[:] for row in matrix], d.copy())
a_gauss = calculate_a(x, h, gauss_solution)
b_gauss = calculate_b(x, h, gauss_solution)

print(gauss_solution)
print(a_gauss)
print(b_gauss)

[0.19999999999999996, 0.47491448152236915, 0.3003420739105226, 0.12371722283553932, 0.4047890347473239, 0.05712663817515884, 0.5667044125520472, 0.0760557116166542, 0.3290727409813211, 0.40765332445807634, -0.1596860388136224, 0.8310908307964004, -0.16467728437197943, 0.42761830669152584, 0.2542040576058802, 0.35556546288493635, 0.12353409085439998, 0.35029817369743826, 0.27527321435583, 0.3486089688792674, 0.1302909101270834, 0.3302273906124161, 0.3487995274233202, 0.07457449969420091, 0.5529024737998933, 0.11381560510625992, 0.19183510577508414, 0.3188439717933353, 0.33278900705164277, 0.14999999999999147]
[0.9541809197462718, 1.429095401268641, 1.7294374751791637, 1.853154698014703, 2.2579437327620266, 2.315070370937186, 2.8817747834892327, 2.957830495105887, 3.286903236087208, 3.6945565605452844, 3.534870521731662, 4.365961352528062, 4.201284068156083, 4.628902374847609, 4.88310643245349, 5.238671895338426, 5.3622059861928255, 5.712504159890264, 5.987777374246094, 6.336386343125361

## Jacobi

In [18]:
jacobi_solution = jacobi(d, lambda_, mi, [0]*len(matrix), 10**-8, 1000)
a_jacob = calculate_a(x, h, jacobi_solution)
b_jacob = calculate_b(x, h, jacobi_solution)

print(jacobi_solution)
print(a_jacob)
print(b_jacob)

[0.19999999999999996, 0.4749144818630316, 0.30034207447765754, 0.12371722376682283, 0.4047890358822926, 0.05712663946864474, 0.5667044142410887, 0.07605571301249817, 0.32907274317184015, 0.4076533257749337, -0.1596860362320856, 0.8310908319884435, -0.16467728156446476, 0.42761830783188703, 0.25420406044961286, 0.3555654640975237, 0.12353409356079453, 0.35029817507899563, 0.27527321679589534, 0.34860897044382005, 0.13029091221931388, 0.33022739226924125, 0.3487995291167185, 0.07457450125873574, 0.5529024750536385, 0.1138156063420154, 0.19183510654948827, 0.3188439724785911, 0.33278900731409566, 0.14999999999999147]
[0.9541809196894947, 1.4290954012308956, 1.7294374751184722, 1.8531546979807554, 2.2579437327356073, 2.31507037087126, 2.8817747835380993, 2.9578304949734413, 3.2869032362328183, 3.6945565603345045, 3.5348705219632444, 4.365961352258817, 4.201284068433942, 4.6289023745637135, 4.883106432725347, 5.238671895089458, 5.362205986413632, 5.712504159713846, 5.987777374392013, 6.3363

## SΔ(t)

Para t no intervalo de \[t<sub>i</sub>, t<sub>i+1</sub>\]



SΔ(t) = <sup>M<sub>i</sub></sup>&frasl;<sub>6h<sub>i+1</sub></sub> (t<sub>i+1</sub> - t)³ + <sup>M<sub>i+1</sub></sup>&frasl;<sub>6h<sub>i+1</sub></sub> (t - t<sub>i</sub>)³ + A<sub>i</sub>(t - t<sub>i</sub>) + B<sub>i</sub>


Resumindo o problema a encontrar a qual intervalo pertence t

In [19]:
def binary_search(arr, value):
    m = 0
    M = len(arr)
    
    while abs(M-m) > 1:
        k = int((M+m)/2)
        
        if value > arr[k]:
            m = k
        else:
            M = k
    
    return m

In [20]:
def s_delta(m, t_, h, t, a, b):
    i = binary_search(t_, t)
    
    first = (t_[i+1]-t) ** 3 * m[i]/(6*h[i+1]) 
    second = (t-t_[i]) ** 3 * m[i+1]/(6*h[i+1]) 
    third = a[i]*(t-t_[i]) + b[i]
    
    return first + second + third

In [21]:
# Numero do grupo
pi = 16
therm = (100-2.3*pi)

m = gauss_solution
a = a_gauss
b = b_gauss

# f(t)
f = lambda t_value: s_delta(m, t, h, t_value, a, b) -therm

O Objetivo é encontrar t tal que f(t) = 0. Para isso, usaremos o método de Newton.

Como ele leva a derivada, abaixo mostra-se a derivada de f em relação a t.

f'(t) = S'Δ(t) = <sup>-M<sub>i</sub></sup>&frasl;<sub>2h<sub>i+1</sub></sub> (t<sub>i+1</sub> - t)² + <sup>M<sub>i+1</sub></sup>&frasl;<sub>2h<sub>i+1</sub></sub> (t - t<sub>i</sub>)² + A<sub>i</sub>

In [22]:
def d_s_delta(m, t_, h, t, a):
    i = binary_search(t_, t)
    
    first = (t_[i+1]-t) ** 2 * (-m[i]/(2*h[i+1]))
    second = (t-t_[i]) ** 2 * m[i+1]/(2*h[i+1])
    third = a[i]
    
    return first + second + third

In [23]:
df = lambda t_value: d_s_delta(m, t, h, t_value, a)

In [24]:
def newton(f, df, init, epsilon, k):
    
    i = 0
    x = init
    while i < k:
        xx = x
        if df(xx) != 0:
            xx -= f(xx)/df(xx)
            err = xx-x
            x = xx
            if(i < 6):
                print((i+1, x))
        else:
            break
            
        if abs(err) <= epsilon:
            break
            
        i += 1
    
    return xx

Para usá-lo, é necessário encontrar um intervalo o qual a raiz esteja. Além disso, para todo o valor do intervalo, sua derivada não pode zerar e sua segunda derivada não pode trocar de sinal no intervalo

In [25]:
init, end = 0, 0
for i in range(len(t)-1):
    if f(t[i]) * f(t[i+1]) < 0:
        init, end = t[i], t[i+1]
        print(f"You should use the interval {t[i], t[i+1]}")

You should use the interval (18, 19)


In [26]:
i = init

while i <= end:
    print(df(i))
    i += 0.1
print(df(i))

5.850140767068178
5.87803476727638
5.906662125029815
5.936022840328484
5.966116913172388
5.996944343561526
6.028505131495899
6.060799276975505
6.093826780000347
6.127587640570423
6.162081858685732


Apesar do conteúdo acima não servir como prova formal, consideraremos que f' nunca é 0 no intervalo \[18, 19\], pois entende-se que f' é crescente

In [27]:
def dd_s_delta(m, t_, h, t, a):
    i = binary_search(t_, t)
    
    first = (t_[i+1]-t) * m[i]/h[i+1]
    second = (t-t_[i]) * m[i+1]/h[i+1]
    
    return first + second

In [28]:
ddf = lambda t_value: dd_s_delta(m, t, h, t_value, a)

i = init

while i <= end:
    print(ddf(i))
    i += 0.1
print(ddf(end))

0.27527321435583
0.2826067898081739
0.2899403652605177
0.2972739407128615
0.30460751616520537
0.31194109161754924
0.31927466706989305
0.3266082425222369
0.3339418179745807
0.3412753934269246
0.3486089688792674


De forma semelhante a f', tomaremos como positivo todo valor de f'' no intervalo \[18, 19\]. Portanto, executando Newton, temos o seguinte. 

In [29]:
newton(f, df, init, 1e-10, 100000)

(1, 18.324778509723313)
(2, 18.322265343514427)
(3, 18.322265184606408)
(4, 18.322265184606408)


18.322265184606408

Somente para efeito de teste, abaixo está o valor de f para o t acima calculado. Deve ser o mais próximo possível de 0

In [34]:
f(newton(f, df, init, 1e-10, 10000))

(1, 18.324778509723313)
(2, 18.322265343514427)
(3, 18.322265184606408)
(4, 18.322265184606408)


-7.105427357601002e-15

## 1 Mostre que A é estritamente diagonal dominante e calcule o valor de σ

Como A é definida, os valores da linha são tais que pertencem ao conjunto {2, λ<sub>i</sub>, μ<sub>i</sub>, 0}, onde, só 0 se repete, para i entre 0 e o número de linhas de A menos 1. Por definição 2 sempre está na diagonal. Quer-se provar que este 2 é maior que qualquer valor de λ e μ somados.

Como μ é definido como esta divisão <sup>h<sub>i</sub></sup>&frasl;<sub>(h<sub>i</sub> + h<sub>i+1</sub>)</sub> e, por sua vez, h = t<sub>i+1</sub> - t<sub>i</sub> e percebendo que a divisão de um número pela adição deste a um segundo só pode ser maior que 1 se este segundo seja negativo, quer-se provar que t é crescente.

De forma simétrica ocorre para λ.

Como t representa contagem de tempo, t é estritamente crescente, implicando que o teto da sequência μ e λ é 1 e portanto o teto da soma de ambos é 2.

In [30]:
abs_matrix = [[abs(el) for el in row] for row in matrix]

sigma = max([
    (1/abs(matrix[i][i])) * (sum(abs_matrix[i]) - abs_matrix[i][i]) 
    for i in range(len(matrix))
])

print(sigma)

0.5


## 2 

Para calcular k, obteremos o primeiro vetor, informado a função que rode uma única vez. Calcularemos sua norma e procuraremos para qual inteiro k, a equação resulta em um valor menor que 10<sup>-8</sup>


In [31]:
y_0_ = [0] * len(t)
y_1_ = jacobi(d, lambda_, mi, [0]*len(t), 10**-8, 1)

norm = norm_of_vectors_difference(y_1_, y_0_)

print(f"\nnorm = {norm}")

k = 0
value = 1

while value > 1e-8:
    k += 1
    value = (sigma ** k)
    value /= (1-sigma)
    value *= norm

print(f"k = {k}")


norm = 0.75
k = 28


## 3

In [32]:
y_star = gauss([row[:] for row in matrix], d.copy())
print(y_star)

[0.19999999999999996, 0.47491448152236915, 0.3003420739105226, 0.12371722283553932, 0.4047890347473239, 0.05712663817515884, 0.5667044125520472, 0.0760557116166542, 0.3290727409813211, 0.40765332445807634, -0.1596860388136224, 0.8310908307964004, -0.16467728437197943, 0.42761830669152584, 0.2542040576058802, 0.35556546288493635, 0.12353409085439998, 0.35029817369743826, 0.27527321435583, 0.3486089688792674, 0.1302909101270834, 0.3302273906124161, 0.3487995274233202, 0.07457449969420091, 0.5529024737998933, 0.11381560510625992, 0.19183510577508414, 0.3188439717933353, 0.33278900705164277, 0.14999999999999147]


## 4

In [33]:
y_jacob = jacobi(d, lambda_, mi, [0]*len(t), 1e-8, 28)

print(y_jacob)
print()
print(f"norm = {norm_of_vectors_difference(y_jacob, y_star)}")

[0.19999999999999996, 0.4749144818630316, 0.30034207447765754, 0.12371722376682283, 0.4047890358822926, 0.05712663946864474, 0.5667044142410887, 0.07605571301249817, 0.32907274317184015, 0.4076533257749337, -0.1596860362320856, 0.8310908319884435, -0.16467728156446476, 0.42761830783188703, 0.25420406044961286, 0.3555654640975237, 0.12353409356079453, 0.35029817507899563, 0.27527321679589534, 0.34860897044382005, 0.13029091221931388, 0.33022739226924125, 0.3487995291167185, 0.07457450125873574, 0.5529024750536385, 0.1138156063420154, 0.19183510654948827, 0.3188439724785911, 0.33278900731409566, 0.14999999999999147]

norm = 2.8437326382579897e-09
